# ConversationEntityMemory


## Overview

```ConversationEntityMemory``` allows the conversation system to retain facts about specific entities mentioned during the dialogue.

It extracts information about entities from the conversation (using an LLM) and 
accumulates knowledge about these entities over time (also using an LLM)


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Entity Memory Conversation Example](#entity-memory-conversation-example)
- [Retrieving Entity Memory](#retrieving-entity-memory)

### References

- [LangChain Python API Reference > langchain: 0.3.13 > memory > ConversationEntityMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.entity.ConversationEntityMemory.html)
----

## Environment Setup
Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.


In [ ]:
## Environment Setup
%%capture --no-stderr
%pip install langchain langchain-opentutorial langchain-community langchain-openai

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ConversationEntityMemory",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it.

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Entity Memory Conversation Example

這個 example 示範如何使用 ```ConversationEntityMemory``` 來儲存和管理在對話中提到的實體的資訊。對話會累積關於這些實體的知識，同時保持自然的流程。


In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory.entity import ConversationEntityMemory

In [3]:
from langchain.prompts import PromptTemplate

entity_memory_conversation_template = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template="""
你是一個由 OpenAI 訓練的大型語言模型所驅動的助手，
你能協助人類完成各種任務，從簡單的問題解答，到深入探討各種主題。

你能夠產生接近人類的自然語言，提供自然流暢的對話體驗與相關且連貫的回應。

你不斷學習與進步，透過大量文字資訊來提供準確、實用的回覆，
並能根據上下文與使用者提供的個人資訊，結合你自身的知識進行輔助與說明。

Context:
{entities}

Current conversation:
{history}
Last line:
Human: {input}
You:
""",
)

print(entity_memory_conversation_template)

input_variables=['entities', 'history', 'input'] input_types={} partial_variables={} template='\n你是一個由 OpenAI 訓練的大型語言模型所驅動的助手，\n你能協助人類完成各種任務，從簡單的問題解答，到深入探討各種主題。\n\n你能夠產生接近人類的自然語言，提供自然流暢的對話體驗與相關且連貫的回應。\n\n你不斷學習與進步，透過大量文字資訊來提供準確、實用的回覆，\n並能根據上下文與使用者提供的個人資訊，結合你自身的知識進行輔助與說明。\n\nContext:\n{entities}\n\nCurrent conversation:\n{history}\nLast line:\nHuman: {input}\nYou:\n'


In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

conversation = ConversationChain(
    llm=llm,
    prompt=entity_memory_conversation_template,
    memory=ConversationEntityMemory(llm=llm),
)

/var/folders/px/22qqx_qn399gggmw1gsvgk000000gn/T/ipykernel_71501/2834047022.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationEntityMemory(llm=llm),
/opt/homebrew/lib/python3.11/site-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/var/folders/px/22qqx_qn399gggmw1gsvgk000000gn/T/ipykernel_71501/2834047022.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


In [5]:
# Input conversation
response = conversation.predict(
    input=(
        "Amelia 是一位屢獲殊榮的風景攝影師，曾走遍世界各地捕捉大自然的奇景。\n",
        "David 則是一位致力於保護瀕危物種的野生動物保育人士。\n",
        "他們正計畫開設一間以大自然為靈感的攝影藝廊與學習中心，並藉此為保育計畫籌募資金。\n"
    )
)

# Print the assistant's response
print(response)

這聽起來是一個非常有意義的計畫！Amelia 的攝影作品可以展示大自然的美麗，吸引更多人關注環境保護，而 David 的保育工作則能夠讓人們了解保護瀕危物種的重要性。這樣的攝影藝廊與學習中心不僅能夠提供藝術欣賞的空間，還能夠成為教育與宣傳的平台，讓更多人參與到保護自然的行動中來。你們有什麼具體的計畫或想法嗎？例如，展覽的主題、課程內容或是如何籌募資金等？


## Retrieving Entity Memory
Let's examine the conversation history stored in memory using the ```memory.entity_store.store``` method to verify memory retention.

In [6]:
# Print the entity memory
conversation.memory.entity_store.store

{'Amelia': 'Amelia 是一位屢獲殊榮的風景攝影師，曾走遍世界各地捕捉大自然的奇景，並正計畫開設一間以大自然為靈感的攝影藝廊與學習中心，藉此為保育計畫籌募資金。',
 'David': 'David 是一位致力於保護瀕危物種的野生動物保育人士，並計畫開設一間以大自然為靈感的攝影藝廊與學習中心，藉此為保育計畫籌募資金。'}